# QuPDE usage examples

QuPDE is a Python library that finds a quadratic transformation (quadratization) for nonquadratic PDEs built using Sympy objects. QuPDE handles spatially one-dimensional PDEs that are polynomial or rational. 

First we import Sympy and QuPDE 

In [1]:
import sympy as sp
from qupde import quadratize

## General usage
### Korteweg-de Vries equation

The Korteweg-de Vries (KdV) equation is a model for the study of weakly nonlinear long waves, incorporating leading nonlinearity and dispersion: 

$$u_t = au^2 u_x - u_{xxx}.$$

Let us find a quadratization for the KdV equation using QuPDE. First, we write the differential equation using Sympy objects:

In [17]:
t, x = sp.symbols("t x")
u = sp.Function("u")(t, x)
a = sp.symbols("a", constant=True)

u_t = a * u**2 * sp.Derivative(u, x) - sp.Derivative(u, x, 3)

To find a quadratization for the PDE we use the main function of the software called *quadratize*. This function receives as input the PDE as a list of tuples, where each tuple represents a differential equation. The first entry of each tuple is an undefined function and the second entry is its corresponding differential equation right-hand side. In our example: 

In [18]:
new_pde = quadratize([(u, u_t)])

This function returns an object with the PDE quadratic transformation that stores the new PDE and the auxiliary variables introduced. We can get the auxiliary variables and the quadratic transformation by running

In [19]:
new_pde.get_aux_vars()

([u**2], [])

In [20]:
new_pde.get_quad_sys()

[Eq(w_0t, a*w_0*w_0x1 + 6*u_x1*u_x2 - w_0x3), Eq(u_t, a*u*w_0x1/2 - u_x3)]

Now, if we want to see the quadratization and the transformed PDE in a more readable format, we call the same function but with the optional *printing* parameter with the available printing options: 
- `'pprint'` for pretty printing (Sympy's functionality).
- `'latex'` for printing the result in LaTeX format. 

In [21]:
quadratize([(u, u_t)], printing="pprint")


Quadratization:
      2
w₀ = u 

Quadratic PDE:
w₀ₜ = a⋅w₀⋅w₀ₓ₁ + 6⋅uₓ₁⋅uₓ₂ - w₀ₓ₃
     a⋅u⋅w₀ₓ₁      
uₜ = ──────── - uₓ₃
        2          


In [22]:
quadratize([(u, u_t)], printing="latex")


Quadratization:
w_{0} = u^{2}

Quadratic PDE:
w_{0t} = a w_{0} w_{0x1} + 6 u_{x1} u_{x2} - w_{0x3}
u_{t} = \frac{a u w_{0x1}}{2} - u_{x3}


## *Quadratize* parameters

The *quadratize* function has additional parameters that the user can modify. These correspond to: 
1. The differential order of the quadratization, which determines determines the maximum spatial derivative order of the original PDE variables (regularity restriction).  
2. The heuristic to sort each set of new variables introduced when searching for an optimal quadratization.
3. The bound of the maximum number of auxiliary variables to explore. 
4. The symbol of the first independent variable (default is *t*). 
5. The maximum order of spatial derivatives of the unknown functions allowed within the auxiliary variables. 
6. The search algorithm.
7. The printing format for displaying the quadratization and the quadratic PDE. 

We offer examples to show how some of them work. 

### 1. Change the differential order of the quadratization
By default, this value is set to three times the maximum order of derivatives found for the unknown functions in the PDE. It is important to note that the order of derivatives allowed directly affects the algorithm's ability to find a quadratization. For example, if we set this value to 0 for the KDV equation, we obtain an unsuccesful search. 

In [ ]:
quadratize([(u, u_t)], diff_ord=0)

In some cases, increasing this parameter will be required to obtain a quadratization. 

### 2. Change sorting heuristic
In the algorithm, there are three heuristics implemented: 
- By order of derivatives and total degree of the monomials (`by_order_degree`)
- By total degree and order of derivatives of the monomials (`by_degree_order`). 
- By the function: $degree + 2 \cdot order$ (`by_fun`).

The default option implemented is `by_fun`. If we want to use the sorting function `by_order_degree` to find a quadratization, we run

In [10]:
quadratize([(u, u_t)], sort_fun="by_order_degree", printing="pprint")


Quadratization:
      2
w₀ = u 

Quadratic PDE:
w₀ₜ = a⋅w₀⋅w₀ₓ₁ - 2⋅u⋅uₓ₃
uₜ = a⋅uₓ₁⋅w₀ - uₓ₃


### 3. Change the bound of maximum number of new variables

The default number for this parameter is 10. If we want to quadratize a PDE that is simple in terms of polynomial degrees, we may want to decrease the bound on the number of variables to find the optimal quadratization faster. Or if a PDE system has higher degree, we may want to increase this bound. To do this, we just arbitrarly set the parameter `nvars_bound`. In our example, if we change this bound to 4, we obtain an optimal quadratization faster:

In [14]:
import time

ti = time.time()
quadratize([(u, u_t)], printing="pprint")
print("Total time with nvars_bound=10: ", time.time() - ti)

ti = time.time()
quadratize([(u, u_t)], nvars_bound=2, printing="pprint")

print("Total time with nvars_bound=2: ", time.time() - ti)


Quadratization:
      2
w₀ = u 

Quadratic PDE:
w₀ₜ = a⋅w₀⋅w₀ₓ₁ - 2⋅u⋅uₓ₃
uₜ = a⋅uₓ₁⋅w₀ - uₓ₃
Total time with nvars_bound=10:  0.10163307189941406

Quadratization:
      2
w₀ = u 

Quadratic PDE:
w₀ₜ = a⋅w₀⋅w₀ₓ₁ - 2⋅u⋅uₓ₃
uₜ = a⋅uₓ₁⋅w₀ - uₓ₃
Total time with nvars_bound=2:  0.08348965644836426


### 4. Change the symbol for the first independent variable

By default, this parameter is *t*. If we want to change the symbol of the first independent variable, we set the parameter `first_indep` equal to the new symbol. Note that changing the symbol for the second independent variable can be done just by defining the equation in Sympy with this new symbol. 

In [17]:
z = sp.symbols("z")
y = sp.symbols("y")
v = sp.Function("v")(z, y)

vz = a * v**2 * sp.Derivative(v, y) - sp.Derivative(v, y, 3)

quadratize([(v, vz)], first_indep=z, printing="pprint")


Quadratization:
      2
w₀ = v 

Quadratic PDE:
w_0z = a⋅w₀⋅w_0y1 + 6⋅v_y1⋅v_y2 - w_0y3
v_z = a⋅v_y1⋅w₀ - v_y3


### 5. Change the maximum order of derivatives of the auxiliary variables

By default, we prune every branch that introduces spatial derivatives of higher order than those within the original equations. To change this parameter, we set `max_der_order` to the desired maximum order. An important note is that in some cases, we need to relax this limit to obtain a quadratization for a PDE. For example, if we run *quadratize* for the equation

In [49]:
ut2 = sp.Derivative(u, x, 2) ** 2 * u
quadratize([(u, ut2)], diff_ord=4, printing="pprint", max_der_order=1)

Quadratization not found


[]

the algorithm does not find a quadratization. Now, if we relax the maximum order of derivatives rule and allow derivatives up to order 2, we obtain a different result 

In [50]:
quadratize([(u, ut2)], diff_ord=4, printing="pprint", max_der_order=2)


Quadratization:
w₀ = u⋅uₓ₂
        2
w₁ = uₓ₁ 

Quadratic PDE:
                                              2
                        2                 w₁ₓ₁ 
w₀ₜ = 2⋅w₀⋅w₀ₓ₂ + 2⋅w₀ₓ₁  - 2⋅w₀ₓ₁⋅w₁ₓ₁ + ─────
                                            2  
                        2
                    w₁ₓ₁ 
w₁ₜ = 2⋅w₀ₓ₁⋅w₁ₓ₁ - ─────
                      2  
uₜ = uₓ₂⋅w₀


### 6. Change the search algorithm 

The default algorithm for searching a quadratization is the branch-and-bound framework. To change this, set the parameter `search_alg` to either `'bnb'` for branch-and-bound, or `'inn'` for the incremental nearest neighbor implementation. 

In [47]:
quadratize([(v, vz)], first_indep=z, search_alg="inn", printing="pprint")


Quadratization:
      2
w₀ = v 

Quadratic PDE:
w_0z = a⋅w₀⋅w_0y1 + 6⋅v_y1⋅v_y2 - w_0y3
v_z = a⋅v_y1⋅w₀ - v_y3


## Other examples
### Allen-Cahn equation

First, we run QuPDE for the Allen-Cahn equation, described by the PDE $$u_t = u_{xx} + u - u^3.$$ This time, we print the result in LaTeX format. 

In [51]:
t, x = sp.symbols("t x")
u = sp.Function("u")(t, x)

u_t = sp.Derivative(u, x, 2) + u - u**3

quadratize([(u, u_t)], printing="latex")


Quadratization:
w_{0} = u^{2}

Quadratic PDE:
w_{0t} = 2 u^{2} + 2 u u_{x2} - 2 w_{0}^{2}
u_{t} = - u w_{0} + u + u_{x2}


### FitzHugh-Nagamo system

The FitzHugh-Nagamo system is a simplified neuron model of the Hodgkin-Huxley model, which describes activation and deactivation dynamics of a spiking neuron. Its governing equations are

$$ v_t = \epsilon v_{xx} + \dfrac{1}{\epsilon}v(v - 0.1)(1 - v) - \dfrac{1}{\epsilon}u + \dfrac{1}{\epsilon}q, $$
$$ u_t = hv - \gamma u + q. $$

We define the symbolic coefficients first to use QuPDE on this equation. 

In [52]:
t, x = sp.symbols("t x")
v = sp.Function("v")(t, x)
y = sp.Function("y")(t, x)
epsilon, h, gamma, r = sp.symbols("epsilon h gamma r", constant=True)

v_t = (
    epsilon * sp.Derivative(v, x, 2)
    - (1 / epsilon) * (v * (v - 0.1) * (1 - v))
    - y / epsilon
    + r / epsilon
)
y_t = h * v - gamma * y + r

quadratize([(v, v_t), (y, y_t)], search_alg="bnb", printing="pprint")


Quadratization:
      2
w₀ = v 

Quadratic PDE:
         2 ⎛     2   w₀ₓ₂⎞           11⋅v⋅w₀               2   w₀
w₀ₜ = 2⋅ε ⋅⎜- vₓ₁  + ────⎟ + 2⋅r⋅v - ─────── - 2⋅v⋅y + 2⋅w₀  + ──
           ⎝          2  ⎠              5                      5 
      2                  v    11⋅w₀    
vₜ = ε ⋅vₓ₂ + r + v⋅w₀ + ── - ───── - y
                         10     10     
yₜ = -γ⋅y + h⋅v + r
